### Fake news detection using Logisctic regression

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavansaipendry/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

What are *stopwords*? 

The words in english which do not contribute much weight to a sentence. For example I, the, it etc.,

In [13]:
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Data Pre-processing

In [14]:
#loading the dataset 
news_dataset = pd.read_csv("/Users/pavansaipendry/Desktop/Master's/Machine Learning/Projects/Fake News Predciction/train.csv")

In [24]:
news_dataset.shape

(20800, 5)

In [17]:
news_dataset.head()

#1 represents fake
#0 represents real

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

We are replacing the *Null* values with empty string

In [26]:
news_dataset = news_dataset.fillna('')

In [27]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

Merging the <b>title</b> and <b>author</b> into a single column and making it as a new feature

In [29]:
news_dataset['content '] = news_dataset['author'] + " " + news_dataset['title']

In [30]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [32]:
news_dataset.value_counts('label')

label
1    10413
0    10387
Name: count, dtype: int64

Seperating the Data and label

In [33]:
X = news_dataset.drop(columns='label' , axis=1)
Y = news_dataset['label']

In [35]:
print(X.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  \
0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1  Ever get the feeling your life circles the rou...   
2  Why the Truth Might Get You Fired October 29, ...   
3  Videos 15 Civilians Killed In Single US Airstr...   
4  Print \nAn Iranian woman has been sentenced to...   

                                            content   
0  Darrell Lucus House Dem Aide: We Didn’t Even S...  
1  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...  
2  Consortiumnews

In [37]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Now, **Stemming:**

Stemming is a text normalization technique used in Natural Language Processing (NLP) to reduce words to their root or base form. The primary purpose of stemming is to group together different forms of a word so they can be analyzed as a single item.

In [38]:
porter_stem = PorterStemmer()

In [39]:
def stemming(content):
    #in the below line, we are only keeping the string from a-z and A-Z and do not want any nums or punctionation
    #and also replacing the nums and punctuations with " " from content
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) 
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [40]:
news_dataset['content '] = news_dataset['content '].apply(stemming)

In [41]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


Again we are sepearing the Data and the label

In [42]:
X = news_dataset['content '].values
Y = news_dataset['label'].values

In [43]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [47]:
print(Y)
Y.shape

[1 0 1 ... 0 1 1]


(20800,)

### Now converting the content strings into numerical data

In [48]:
#TF(text freq) assigns count to the repreated words
#idf (inverse freq) if any words doesn't have any meaning but has count, then it will be removed

vectorizer = TfidfVectorizer() 
vectorizer.fit(X)

X = vectorizer.transform(X)

In [52]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

### Train Test Split

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2 , stratify=Y , random_state=2)

### Training the Logistice Regression Model

In [55]:
model = LogisticRegression()

In [56]:
model.fit(X_train, y_train)

LogisticRegression()

### Evaulation - Accuracy score

Accuracy score on Trained data

In [57]:
X_train_prediction = model.predict(X_train)
training_data_acc = accuracy_score(X_train_prediction , y_train)

In [58]:
print("Accuarcy score of Train data" , training_data_acc)

Accuarcy score of Train data 0.9865985576923076


Accuarcy score on test data

In [59]:
X_test_prediction = model.predict(X_test)
test_data_acc = accuracy_score(X_test_prediction , y_test)

In [60]:
print("Accuarcy score of test data" , test_data_acc)

Accuarcy score of test data 0.9790865384615385


### A Predictive System based on Trained and Tested data

In [61]:
new_data_for_prediction = X_test[0]

prediction = model.predict(new_data_for_prediction)
print("Predicted Data" , prediction)

if prediction[0] == 0:
    print("Real News")
else:
    print("Fake news")

Predicted Data [1]
Fake news


Summary of the Project
1. First imported all the needed dependencies
2. Then, we made a new column called "content", it is the mergred column of author and title
3. Later, we pre-procssed the data by stemming, and then converting the string type into numerical data with the help of TfidfVectorizer() 
4. Then, we split the data into train_test_split. 
5. Then trained the model using Logistice regression
6. After model fit, we checked the accuracy on trained_data. The accuracy on trained_data is 0.9865985576923076.The accuracy on test_data is 0.0.9790865384615385. 
7. Atlast, we made a predictive system